This is my most up to date and as annotated as possible script. For the data set, have it in you local directory (It's uploaded to github too)

In [1]:
#import dependences

import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
from sklearn import preprocessing, cross_validation
from sklearn.neural_network import MLPClassifier


/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def make_network(FILENAME,sklearn=True,keras=False):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    from sklearn.neural_network import MLPClassifier
    
    """
    Given the csv input of all the box scores, arrange it such that the home and away teams are lined up, 
    unnecessary columns removed, and hot encoding is done. Other stuff too probably. Such as normalization, but I 
    didn't do that!
    
    Note that this data has already been doctored from its original form, taking out most unnecessary columns but
    those could be useful later on.
    
    
    Parameters
    ----------
    FILENAME : file
        The csv of the data.
        
    Returns
    -------
    
    
    model : object
        MLP which can predict the outcome of NBA games
        
    """
    #Read in file, remove attempted and # and only account for % since that's more predictive in nature. 
    #*retrospectively that doesn't make sense, could be worth changing!
    data = read_csv(FILENAME) 
    data['3P%'] = np.divide(data['3P'].values,data['3PA'].values) 
    del data['3P'],data['3PA']
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    del data['FG'],data['FGA']
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['FT'],data['FTA']
    data = get_dummies(data)
    del data['VENUE_Home'],data['VENUE_Road']
    #print(data)
    

    dat = []
    
    #reshape the dataset so now each colummn has roadstats and homestats concatenated into the same row, used for NN 
    
    for i in range(len(data.values)):
        data.values[i] = np.reshape(data.values[i],newshape=[1,len(data.values[i])])
    for p in range(int(len(data.values)/2)):
        fullboxgame = np.concatenate((data.values[2*p],data.values[(2*p)+1]))
        dat.append(fullboxgame)
    
    #convert list to array, now possible to array operations previously not possible
    dat = np.array(dat)   
    
    #now to find out the score of the game, and whether or not the spread was covered. 
    roadpts = dat[:,8] #column of all the points scored by road team 
    homepts = dat[:,21] #vice versa of above
    
    endspreadS = roadpts-homepts  #all the final spreads of the game
    openingspreadS = dat[:,9]  #what the predicted spread of ther game was. 
    y = []
    
    #Now I iterated over all these, and hot encoded the labels of each to see whether or not the spread was covered
    #and by what team. 
    for j in range(len(endspreadS)):  
        openspread = openingspreadS[j]
       # print("this is the spread of the road team " + str(openspread))
        endspread = endspreadS[j]
       # print("the road team won by  .. " + str(endspread))
       # if endspread>openspread:
        #    y.append(np.array([0,1,0]))  #OK, now make sure this is formateed properly!
        if openspread + endspread <0:
            y.append(np.array([0,1,0]))  #hoem team covered
        elif openspread + endspread >0:
            y.append(np.array([1,0,0]))  #road covered
        else: 
            y.append(np.array([0,0,1]))  #push!



    y = np.array(y)  #same explanation as above
    X1 = np.concatenate((dat[:,0:8],dat[:,10:21]),axis=1)  #reshaping arrays,
    #since everything got out of order I have to mash it together myself. 
    X = np.concatenate((X1,dat[:,23:26]),axis=1)    #need to go one further column to snag HFT% 
    X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.27)
    #print((X[0]))
    #print(np.shape(X[0]))

    #now to construct a model 
    if sklearn: 
        model = MLPClassifier()
        model.shuffle = True
        model.batch_size = 25
    #model.n_layers_ = 1000000
    #model.n_outputs_= 1000000
    #These don't do anything, have to adjust the layers in some different way! Keras is useful for this.
        model.fit(X_train,y_train)
        print(model.score(X_test,y_test))
    if keras:
        print("keras NN goes here")
    
    
    return model
    

The next function I have is for turning the current nba team statistics (either over the entire season or some stretch) into an array of the same shape and information as the one used for the box scores above. 

In [3]:
def make_prediction_data(filename):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    data = read_csv(filename)
    #convert to per game stats and sort columns. 
    data['ORB'] =  np.divide(data['ORB'].values,data['G'].values)
    data['DRB'] =  np.divide(data['DRB'].values,data['G'].values)
    data['TRB'] =  np.divide(data['TRB'].values,data['G'].values)
    data['AST'] =  np.divide(data['AST'].values,data['G'].values)
    data['STL'] =  np.divide(data['STL'].values,data['G'].values)
    data['BLK'] =  np.divide(data['BLK'].values,data['G'].values)
    data['TOV'] =  np.divide(data['TOV'].values,data['G'].values)
    data['PF'] =  np.divide(data['PF'].values,data['G'].values)
    teams  = data['Team']

    data = data[['ORB' , 'DRB'  ,'TRB' ,  'AST' , 'PF' , 'STL' , 'TOV' , 'BLK' ,'3P%','FG%' ,'FT%']]
    print("Here is every teams index value: ")
    print(teams)
    return teams,data

In [4]:
def game_maker(roadteam,hometeam):
    """After creating a properly formated table, this concats the desired teams so they can be predicted. 
        Based on get team index # based on output of predictor, and make it the input for stats ie GSW are stats[0].
        and so on!
    """
    game = np.concatenate((roadteam,hometeam))
    return game

In [5]:
filetouse = 'teamstats_322.csv'  #downloaded from basketball reference, and specified date 

teams, stats =make_prediction_data('teamstats_322.csv')

Here is every teams index value: 
0     Golden State Warriors*
1     Toronto Raptors*      
2     Houston Rockets*      
3     New Orleans Pelicans  
4     Minnesota Timberwolves
5     Denver Nuggets        
6     Cleveland Cavaliers   
7     Oklahoma City Thunder 
8     Los Angeles Clippers  
9     Charlotte Hornets     
10    Brooklyn Nets         
11    Washington Wizards    
12    Indiana Pacers        
13    Los Angeles Lakers    
14    Philadelphia 76ers    
15    Portland Trail Blazers
16    Milwaukee Bucks       
17    New York Knicks       
18    Phoenix Suns          
19    Miami Heat            
20    Orlando Magic         
21    Boston Celtics*       
22    Atlanta Hawks         
23    San Antonio Spurs     
24    Chicago Bulls         
25    Utah Jazz             
26    Detroit Pistons       
27    Dallas Mavericks      
28    Sacramento Kings      
29    Memphis Grizzlies     
30    League Average        
Name: Team, dtype: object


In [12]:
#For this spreadsheet, see github. 
nbapredictor = make_network('1517_boxscores_withspreads.csv')

0.7447108603667136


In [13]:
game1 = game_maker(stats.values[1],stats.values[18])
game2 = game_maker(stats.values[3],stats.values[15])
game3 = game_maker(stats.values[7],stats.values[13])

In [14]:
nbapredictor.predict(np.array([game1,game2,game3]))

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])